In [ ]:
import openai, json, requests
client = openai.OpenAI()
from openai.types.chat import ChatCompletionMessage
messages = []

url = "https://nomad-movies.nomadcoders.workers.dev/movies"

def get_popular_movie(**arguments):
    response = requests.get(url)
    movies = response.json()
    #10개만 출력하도록 수정
    return [{"id":m.get("id"),"title":m.get("title")} for m in movies[:10]]

def get_movie_details(id):
    response = requests.get(url + "/" + id)
    movies_details = response.json()
    return movies_details

def get_similar_movies(movie_name):
    response = get_popular_movie()
    id = next((movie['id'] for movie in response if movie['title'] == movie_name), None)
    response = requests.get(url + "/"+ str(id) + "/similar")
    movies = response.json()
    return [ {"id": m.get("id"), "title":m.get("title")}for m in movies ]

FUNCTION_MAP = {"get_popular_movie":get_popular_movie, "get_movie_details": get_movie_details, "get_similar_movies": get_similar_movies}

TOOLS = [ #함수 타입을 만들어줌
    {
        "type":"function",
        "function": {
            "name":"get_popular_movie",
            "description":(
                "A function that retrieves the results by searching for famous movies at 'https://nomad-movies.nomadcoders.workers.dev/movies'. The result is output as follows. (Example: List of currently popular movies. 1. god father (id number), 2. brave heart (id number) 3...)"           
                           ),
        }
    },
    {
        "type":"function",
        "function": {
            "name":"get_movie_details",
            "description":"This function takes a movie ID number as input and outputs detailed information about the movie with the specific ID as 'https://nomad-movies.nomadcoders.workers.dev/movies/{id}'. (Example: get_movie_details(500)) The output results include all information such as the movie title, movie ID, original_language, etc.",
            "parameters": {
                "type":"object",
                "properties":{
                    "id":{
                        "type":"string",
                        "description":"This is the id number that will be used to view detailed information about the movie.",
                    }
                },
                "required":["id"]
            }
        }
    },
    {
        "type":"function",
        "function": {
            "name":"get_similar_movies",
            "description":"This function takes a movie ID number as input and outputs information about movies similar to the specific ID as 'https://nomad-movies.nomadcoders.workers.dev/movies/{id}/similar'. (Example: get_similar_movies_(500)) The output results are the movie ID and movie title.",
            "parameters": {
                "type":"object",
                "properties":{
                    "movie_name":{
                        "type":"string",
                        "description":"This is to find movies of a similar type to the movies of this id.",
                    }
                },
                "required":["id"]
            }
        }
    }
]



def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append({ #이작업은 messages에 tool_calls를 저장하기 위한것
            "role":"assistant",
            "content": message.content or "", #message의 content가 있으면 넣고 아니면 비워두기
            "tool_calls":[{
                "id": tool_call.id,
                "type":"function",
                "function":{
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments,
                }
            } 
            for tool_call in message.tool_calls #당연히 message에서 가지고 왔기 때문에 살아있음.
            ],
        })
        
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments  = tool_call.function.arguments or "{}"
            
            print(f"Agent: [{function_name}]({arguments}) 호출]")
            
            try:
                args = json.loads(arguments)
                if not isinstance(args, dict):
                    args = {}
            except json.JSONDecodeError:
                args = {}
            
            function_to_run = FUNCTION_MAP.get(function_name)
            result = function_to_run(**args) #키값을 이용해서 value를 찾아서 넣음.
            tool_content=json.dumps(result, ensure_ascii=False)
            print(f"Ran {function_name} with args {args} for a result of {tool_content}")
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name":function_name,
                "content":tool_content
            })
        call_ai() #더이상 tool_calls가 없으면 지금까지 messages에 데이터가 더해져서 아래쪽 else로 들어가서 한 번 더 결과를 만들어 내고 이것을 그대로 AI에다가 저장하는 것.
    else:
        messages.append({"role":"assistant", "content":message.content}) #최초 입력한 것
        print(f"Agent: {message.content}")

def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages,
        tools = TOOLS #어떤 도구를 이용해서 calling할지 확인함.
    )
    process_ai_response(response.choices[0].message) #안에 tools call이 포함되어 있음.            
while True:
    message = input("메시지를 입력해줘")
    if message == "quit" or message =="q":
        break
    else:
        messages.append({"role":"user", "content":message})
        print(f"user: {message}")
        call_ai()

user: 지금 인기있는 영화를 알려줘
Agent: [get_popular_movie]({}) 호출]
Ran get_popular_movie with args {} for a result of [{"id": 1290821, "title": "Shelter"}, {"id": 1236153, "title": "Mercy"}, {"id": 799882, "title": "The Bluff"}, {"id": 1244975, "title": "The Orphans"}, {"id": 1088434, "title": "Hellfire"}, {"id": 1301306, "title": "A Woman Scorned"}, {"id": 1523145, "title": "Your Heart Will Be Broken"}, {"id": 680493, "title": "Return to Silent Hill"}, {"id": 1272837, "title": "28 Years Later: The Bone Temple"}, {"id": 1419406, "title": "The Shadow's Edge"}]
Agent: 현재 인기 있는 영화 목록은 다음과 같습니다:

1. Shelter (ID: 1290821)
2. Mercy (ID: 1236153)
3. The Bluff (ID: 799882)
4. The Orphans (ID: 1244975)
5. Hellfire (ID: 1088434)
6. A Woman Scorned (ID: 1301306)
7. Your Heart Will Be Broken (ID: 1523145)
8. Return to Silent Hill (ID: 680493)
9. 28 Years Later: The Bone Temple (ID: 1272837)
10. The Shadow's Edge (ID: 1419406)

더 많은 정보가 필요하시면 말씀해 주세요!
user: Mercy에 대해 더 알려줘
Agent: [get_movie_details]({"id":"1